In [ ]:
# import required modules
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
%matplotlib inline

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [ ]:
# read data and fetch Time Series 'Charles Dickens' 
train = pd.read_csv('../input/train_1.csv').fillna(0)
df = None
for i, row in train.iterrows():
    if row['Page'] == "Charles_Dickens_en.wikipedia.org_desktop_all-agents":
        df = row
        break
df.drop(['Page'], inplace=True)
df.index = pd.to_datetime(df.index, format='%Y-%m-%d')

In [ ]:
df.head()

In [ ]:
df.tail()

so our time series is between 1st July, 2015 and 31st December, 2016.     
Now let us plot time series

In [ ]:
import matplotlib.dates as mdates
fig = plt.figure()
ax = fig.add_subplot(1,1,1)  
plt.plot(df)
ax.xaxis.set_major_locator(mdates.DayLocator(interval=90))
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y:%m'))
plt.xlabel('Date')
plt.ylabel('Number of Hits')
plt.title('Wikipedia Hits for Charles Dickens')

we have some peaks in series, let us find dates for these peaks

In [ ]:
df.sort_values()[-5:]

so we have 2 peaks. One is around 19th December, 2016 and other is around 1st December, 2015.




Charles Dickens publishes “A Christmas Carol” on 19th December. 1843. Sudden peak in 2016 seems to be due to this event  

https://www.eventbrite.co.uk/e/cliveden-special-event-19th-december-christmas-carol-concert-with-the-queens-six-sold-out-tickets-27932442688

Below event was organized on December, 2015 which seems to results in high hits around that time          
https://londonvisitors.wordpress.com/2015/12/01/review-christmas-at-the-charles-dickens-museum-1st-december-2015-to-6th-january-2016/

## TIME SERIES FORECASTING

I train my time series on first half of 2016 and forecast on second Half by ARIMA model

In [ ]:
df_train = df['2016-01-01':'2016-06-30'].copy()
df_test = df['2016-07-01':'2016-12-31'].copy()

In [ ]:
from statsmodels.tsa.arima_model import ARIMA
from pandas.plotting import autocorrelation_plot

In [ ]:
autocorrelation_plot(df_train)

In [ ]:
# series seems to be already stationary so no need for difference 
arima = ARIMA(np.array(df_train), [5, 0, 0])
result = arima.fit(disp=False)

In [ ]:
# use the model to predict second half of 2016
ans = result.predict(start=183, end=365).astype(int)

In [ ]:
# better way?
for i in range(len(ans)):
    df_test.iloc[i] = ans[i]

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(1,1,1)  
ax.xaxis.set_major_locator(mdates.DayLocator(interval=90))
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y:%m'))
plt.plot(df)
plt.plot(df_test)